# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Opening a new Business Park in Dubai, UAE

- Build a dataframe of communities in Duba, UAE by web scraping the data from Wikipedia page
- Get the geographical coordinates of the communities
- Obtain the venue data for the communities from Foursquare API
- Explore and cluster the communities
- Select the best cluster to open a new Business Park

In [55]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#pip install geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [2]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

print('Geopy imported.')

Geopy imported.


In [3]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Folium imported.')

Folium imported.


#### Scrap data from Wikipedia page into a DataFrame to obtian communities in Dubai

In [4]:
data_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_communities_in_Dubai")
dubai_df_raw = data_df[0]
dubai_df_raw.columns = ['Community_no','Community','Community_arabic','Area','Population','Density']
dubai_df_raw

,Community_no,Community,Community_arabic,Area,Population,Density
0,126.0,Abu Hail,أبو هيل,1.27 km²,21414,"16,861.4/km²"
1,711.0,Al Awir First,العوير الأولى,NaN,NaN,NaN
2,721.0,Al Awir Second,العوير الثانية,NaN,NaN,NaN
3,333.0,Al Bada,البدع,0.82 km²,18816,22946/km²
4,122.0,Al Baraha,البراحة,1.104 km²,7823,"7,086/km²"
5,373.0,Al Barsha First,البرشاء الأولى,NaN,NaN,NaN
6,376.0,Al Barsha Second,البرشاء الثانية,NaN,NaN,NaN
7,671.0,Al Barsha South First,البرشاء جنوب الاولى,NaN,NaN,NaN
8,672.0,Al Barsha South Second,البرشاء جنوب الثانية,NaN,NaN,NaN
9,673.0,Al Barsha South Third,البرشاء جنوب الثالثة,NaN,NaN,NaN


In [6]:
#Finding datatypes of above dataframe
dubai_df_raw.dtypes

Community_no        float64
Community            object
Community_arabic     object
Area                 object
Population           object
Density              object
dtype: object

#### Creating a new dataframe with all communities in Dubai, UAE

In [7]:
dubai_data = dubai_df_raw[['Community']]
dubai_data

,Community
0,Abu Hail
1,Al Awir First
2,Al Awir Second
3,Al Bada
4,Al Baraha
5,Al Barsha First
6,Al Barsha Second
7,Al Barsha South First
8,Al Barsha South Second
9,Al Barsha South Third


In [8]:
dubai_data.shape

(131, 1)

#### Get the geographical coordinates of all communities in Dubai

In [9]:
def get_latlng(community):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Dubai'.format(community))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
coords = [get_latlng(community) for community in dubai_data['Community'].tolist()]

In [11]:
coords

[[25.28308000000004, 55.33435000000003],
 [25.18593000000004, 55.54126000000008],
 [25.167920000000038, 55.543310000000076],
 [25.21861000000007, 55.26406000000003],
 [25.282800000000066, 55.31678000000005],
 [25.11483000000004, 55.19136000000003],
 [25.107230000000072, 55.20485000000008],
 [25.08958000000007, 55.23424000000006],
 [25.077390000000037, 55.24267000000003],
 [25.062290000000075, 55.23995000000008],
 [25.093420000000037, 55.19044000000008],
 [25.269250000000056, 55.29944000000006],
 [25.272170000000074, 55.30157000000003],
 [25.243370000000027, 55.352670000000046],
 [25.269510000000025, 55.30884000000003],
 [25.25696000000005, 55.30246000000005],
 [25.29871000000003, 55.33546000000007],
 [25.237130000000036, 55.27707000000004],
 [25.220540000000028, 55.34166000000005],
 [25.233420000000024, 55.29001000000005],
 [25.245290000000068, 55.30364000000003],
 [25.27177000000006, 55.33762000000007],
 [25.24282000000005, 55.48440000000005],
 [25.22784000000007, 55.522320000000036],

In [12]:
#Creating coordinates dataframe
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
df_coords.head()

,Latitude,Longitude
0,25.28308,55.33435
1,25.18593,55.54126
2,25.16792,55.54331
3,25.21861,55.26406
4,25.28280,55.31678


In [14]:
df_coords.shape

(131, 2)

#### Merging Dubai communities and coordinates dataframe.

In [15]:
dubai_data = pd.concat([dubai_data, df_coords], axis=1)

In [16]:
dubai_data

,Community,Latitude,Longitude
0,Abu Hail,25.283080,55.334350
1,Al Awir First,25.185930,55.541260
2,Al Awir Second,25.167920,55.543310
3,Al Bada,25.218610,55.264060
4,Al Baraha,25.282800,55.316780
5,Al Barsha First,25.114830,55.191360
6,Al Barsha Second,25.107230,55.204850
7,Al Barsha South First,25.089580,55.234240
8,Al Barsha South Second,25.077390,55.242670
9,Al Barsha South Third,25.062290,55.239950


In [17]:
dubai_data.shape

(131, 3)

#### Creating a map to vizualize communities in Dubai, UAE

In [18]:
# get the coordinates of Dubai
address = 'Dubai'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dubai, UAE {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dubai, UAE 25.0659637, 55.1713403.


In [56]:
# create map of Dubai using latitude and longitude values
map_dubai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, community in zip(dubai_data['Latitude'], dubai_data['Longitude'], dubai_data['Community']):
    label = '{}'.format(community)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dubai)  
    
map_dubai

#### Use the Foursquare API to explore the communities of Dubai

In [20]:
CLIENT_ID = 'G5M1TYVFDKGKTQZORVIXY1SXU0U0BDRZ0W0ZELYURBZ3MTDV' # your Foursquare ID
CLIENT_SECRET = 'QDWINAULBSVQAXREPMK24E5M5E1BORBBJ4V4TJIIYTPN10M5' # your Foursquare Secret
VERSION = '20200605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G5M1TYVFDKGKTQZORVIXY1SXU0U0BDRZ0W0ZELYURBZ3MTDV
CLIENT_SECRET:QDWINAULBSVQAXREPMK24E5M5E1BORBBJ4V4TJIIYTPN10M5


#### Now, let's get the top 100 venues that are within a radius of 500 meters.

In [21]:
# Get the top 100 venues within a radius of 500 meters
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

venues = []

for lat, long, community in zip(dubai_data['Latitude'], dubai_data['Longitude'], dubai_data['Community']):
    
    # create URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            community,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

#### Converting Venues list into a DataFrame

In [22]:
# convert the venues list into a new DataFrame
dubai_venues = pd.DataFrame(venues)

# define the column names
dubai_venues.columns = ['Community', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(dubai_venues.shape)
dubai_venues.head()

(2078, 7)


,Community,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abu Hail,25.28308,55.33435,Habib Bakery,25.281124,55.332774,Bakery
1,Abu Hail,25.28308,55.33435,Mubasher Cafeteria,25.282622,55.336252,Cafeteria
2,Abu Hail,25.28308,55.33435,Pizza Hut,25.282107,55.335684,Pizza Place
3,Abu Hail,25.28308,55.33435,Bait Al Jinnie Junction,25.280546,55.330471,Scenic Lookout
4,Abu Hail,25.28308,55.33435,Hamriya Park,25.285710,55.333000,Park


#### Count of venues in each neighborhood

In [23]:
dubai_venues.groupby('Community').count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Community,,,,,,
Abu Hail,6,6,6,6,6,6
Al Awir First,1,1,1,1,1,1
Al Bada,11,11,11,11,11,11
Al Baraha,9,9,9,9,9,9
Al Barsha First,57,57,57,57,57,57
Al Barsha Second,24,24,24,24,24,24
Al Barsha South First,4,4,4,4,4,4
Al Barsha South Second,4,4,4,4,4,4
Al Barsha Third,5,5,5,5,5,5


In [24]:
dubai_venues.groupby('Community').count().shape

(120, 6)

#### Total Unique Categories in Dubai communities

In [25]:
print('There are total {} uniques categories.'.format(len(dubai_venues['VenueCategory'].unique())))

There are total 241 uniques categories.


#### Analyze each neighbourhood

##### Performing One Hot Encoding on Venues dataframe

In [26]:
# one hot encoding
dubai_onehot = pd.get_dummies(dubai_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dubai_onehot['Community'] = dubai_venues['Community'] 

fixed_columns = [dubai_onehot.columns[-1]] + list(dubai_onehot.columns[:-1])
dubai_onehot = dubai_onehot[fixed_columns]

dubai_onehot.head()

,Community,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Campground,Castle,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cruise,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Laundromat,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Nightlife,Pakistani Restaurant,Park,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Track,Train Station,Tram Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Abu Hail,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [27]:
dubai_onehot.shape

(2078, 242)

#### Grouping rows by community and by taking the mean of the frequency of occurrence of each category

In [28]:
dubai_grouped = dubai_onehot.groupby('Community').mean().reset_index()
dubai_grouped

,Community,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Campground,Castle,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cruise,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Laundromat,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Nightlife,Pakistani Restaurant,Park,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Track,Train Station,Tram Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Abu Hail,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [29]:
dubai_grouped.shape

(120, 242)

#### Print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for com in dubai_grouped['Community']:
    print("----"+com+"----")
    temp = dubai_grouped[dubai_grouped['Community'] == com].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abu Hail----
            venue  freq
0            Park  0.17
1     Pizza Place  0.17
2          Bakery  0.17
3       Cafeteria  0.17
4  Scenic Lookout  0.17


----Al Awir First----
             venue  freq
0             Café   1.0
1              ATM   0.0
2        Multiplex   0.0
3       Nail Salon   0.0
4  Nature Preserve   0.0


----Al Bada----
         venue  freq
0         Café  0.27
1  Coffee Shop  0.18
2     Boutique  0.09
3        Beach  0.09
4       Bakery  0.09


----Al Baraha----
                       venue  freq
0                      Hotel  0.22
1       Fast Food Restaurant  0.11
2                       Café  0.11
3                     Lounge  0.11
4  Middle Eastern Restaurant  0.11


----Al Barsha First----
                       venue  freq
0                      Hotel  0.18
1            Bed & Breakfast  0.07
2                       Pool  0.05
3  Middle Eastern Restaurant  0.05
4       Pakistani Restaurant  0.05


----Al Barsha Second----
                       venue

              venue  freq
0  Department Store  0.25
1       Supermarket  0.25
2    Ice Cream Shop  0.25
3             Hotel  0.25
4               ATM  0.00


----Al Qusais Industrial Fifth----
                    venue  freq
0           Moving Target  0.25
1  Furniture / Home Store  0.25
2               Beach Bar  0.25
3             Pet Service  0.25
4            Perfume Shop  0.00


----Al Qusais Industrial First----
                  venue  freq
0  Fast Food Restaurant  0.50
1           Karaoke Bar  0.25
2       Other Nightlife  0.25
3    Persian Restaurant  0.00
4            Nail Salon  0.00


----Al Qusais Industrial Fourth----
                       venue  freq
0                       Café  0.22
1  Middle Eastern Restaurant  0.11
2                Pizza Place  0.11
3                  Pool Hall  0.11
4       Fast Food Restaurant  0.11


----Al Qusais Industrial Second----
            venue  freq
0            Café  0.16
1     Coffee Shop  0.12
2  Ice Cream Shop  0.12
3      Restauran

                     venue  freq
0      American Restaurant  0.08
1            Shopping Mall  0.08
2              Coffee Shop  0.08
3             Dessert Shop  0.04
4  Comfort Food Restaurant  0.04


----Jumeira Second----
                       venue  freq
0                       Café  0.22
1             Ice Cream Shop  0.06
2                Coffee Shop  0.06
3  Middle Eastern Restaurant  0.06
4             Massage Studio  0.03


----Jumeira Third----
                venue  freq
0       Grocery Store  0.29
1  Italian Restaurant  0.14
2        Tennis Court  0.14
3                Café  0.14
4         Video Store  0.14


----Marsa Dubai----
         venue  freq
0   Restaurant  0.08
1         Café  0.08
2        Hotel  0.06
3  Coffee Shop  0.05
4        Beach  0.05


----Mirdif----
                       venue  freq
0                Coffee Shop  0.14
1                       Café  0.11
2  Middle Eastern Restaurant  0.11
3                        Gym  0.08
4             Cosmetics Shop  0.05


#### Putting above into dataframe

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Community']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
community_venues_sorted = pd.DataFrame(columns=columns)
community_venues_sorted['Community'] = dubai_grouped['Community']

for ind in np.arange(dubai_grouped.shape[0]):
    community_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dubai_grouped.iloc[ind, :], num_top_venues)

community_venues_sorted

,Community,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,Park,Scenic Lookout,Cafeteria,Bakery,Pizza Place,Hookah Bar,Flower Shop,Fish Market,Flea Market,Food
1,Al Awir First,Café,Yoga Studio,Dance Studio,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
2,Al Bada,Café,Coffee Shop,Gym / Fitness Center,Restaurant,Beach,Shopping Plaza,Bakery,Boutique,Electronics Store,English Restaurant
3,Al Baraha,Hotel,Indian Restaurant,Turkish Restaurant,Café,Middle Eastern Restaurant,Lounge,Fast Food Restaurant,American Restaurant,Food Court,Football Stadium
4,Al Barsha First,Hotel,Bed & Breakfast,Pakistani Restaurant,Pool,Indian Restaurant,Middle Eastern Restaurant,Restaurant,Grocery Store,Dessert Shop,Turkish Restaurant
5,Al Barsha Second,Hotel,Indian Restaurant,Coffee Shop,Ice Cream Shop,Park,Chinese Restaurant,Bakery,Tennis Court,Seafood Restaurant,Gym
6,Al Barsha South First,Bakery,Pharmacy,Grocery Store,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop
7,Al Barsha South Second,Café,Supermarket,Office,Gym,Yoga Studio,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
8,Al Barsha Third,Fast Food Restaurant,Art Gallery,Bakery,Convenience Store,Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food
9,Al Buteen,Market,Shopping Mall,Jewelry Store,Art Gallery,Fast Food Restaurant,Pakistani Restaurant,Farmers Market,Flower Shop,Electronics Store,Juice Bar


### Clustering the communities

#### Dropping Community Column from the community venues dataframe

In [33]:
dubai_grouped_clustering = dubai_grouped.drop('Community', 1)
dubai_grouped_clustering.head()

,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Campground,Castle,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cruise,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Laundromat,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Nightlife,Pakistani Restaurant,Park,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Track,Train Station,Tram Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.

In [34]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dubai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([4, 4, 4, 1, 1, 1, 2, 4, 4, 4, 4, 2, 1, 4, 4, 4, 4, 4, 1, 4, 1, 1,
       4, 1, 4, 4, 4, 4, 1, 4, 1, 1, 1, 0, 1, 4, 4, 4, 4, 4, 4, 4, 1, 4,
       4, 4, 4, 4, 4, 4], dtype=int32)

#### Create dataframe top 10 venues and cluster labels 

In [35]:
# add clustering labels
community_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dubai_merged = dubai_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dubai_merged = dubai_merged.join(community_venues_sorted.set_index('Community'), on='Community')

dubai_merged.head() # check the last columns!

,Community,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,25.28308,55.33435,4.0,Park,Scenic Lookout,Cafeteria,Bakery,Pizza Place,Hookah Bar,Flower Shop,Fish Market,Flea Market,Food
1,Al Awir First,25.18593,55.54126,4.0,Café,Yoga Studio,Dance Studio,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
2,Al Awir Second,25.16792,55.54331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Al Bada,25.21861,55.26406,4.0,Café,Coffee Shop,Gym / Fitness Center,Restaurant,Beach,Shopping Plaza,Bakery,Boutique,Electronics Store,English Restaurant
4,Al Baraha,25.28280,55.31678,1.0,Hotel,Indian Restaurant,Turkish Restaurant,Café,Middle Eastern Restaurant,Lounge,Fast Food Restaurant,American Restaurant,Food Court,Football Stadium


In [36]:
dubai_merged

,Community,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,25.283080,55.334350,4.0,Park,Scenic Lookout,Cafeteria,Bakery,Pizza Place,Hookah Bar,Flower Shop,Fish Market,Flea Market,Food
1,Al Awir First,25.185930,55.541260,4.0,Café,Yoga Studio,Dance Studio,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
2,Al Awir Second,25.167920,55.543310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Al Bada,25.218610,55.264060,4.0,Café,Coffee Shop,Gym / Fitness Center,Restaurant,Beach,Shopping Plaza,Bakery,Boutique,Electronics Store,English Restaurant
4,Al Baraha,25.282800,55.316780,1.0,Hotel,Indian Restaurant,Turkish Restaurant,Café,Middle Eastern Restaurant,Lounge,Fast Food Restaurant,American Restaurant,Food Court,Football Stadium
5,Al Barsha First,25.114830,55.191360,1.0,Hotel,Bed & Breakfast,Pakistani Restaurant,Pool,Indian Restaurant,Middle Eastern Restaurant,Restaurant,Grocery Store,Dessert Shop,Turkish Restaurant
6,Al Barsha Second,25.107230,55.204850,1.0,Hotel,Indian Restaurant,Coffee Shop,Ice Cream Shop,Park,Chinese Restaurant,Bakery,Tennis Court,Seafood Restaurant,Gym
7,Al Barsha South First,25.089580,55.234240,2.0,Bakery,Pharmacy,Grocery Store,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop
8,Al Barsha South Second,25.077390,55.242670,4.0,Café,Supermarket,Office,Gym,Yoga Studio,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
9,Al Barsha South Third,25.062290,55.239950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Adding cluster label 5.0 to the Communities with no venues, to make it the 6th cluster

In [37]:
dubai_merged['Cluster Labels'] = dubai_merged['Cluster Labels'].fillna(5.0)

In [38]:
dubai_merged

,Community,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,25.283080,55.334350,4.0,Park,Scenic Lookout,Cafeteria,Bakery,Pizza Place,Hookah Bar,Flower Shop,Fish Market,Flea Market,Food
1,Al Awir First,25.185930,55.541260,4.0,Café,Yoga Studio,Dance Studio,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
2,Al Awir Second,25.167920,55.543310,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Al Bada,25.218610,55.264060,4.0,Café,Coffee Shop,Gym / Fitness Center,Restaurant,Beach,Shopping Plaza,Bakery,Boutique,Electronics Store,English Restaurant
4,Al Baraha,25.282800,55.316780,1.0,Hotel,Indian Restaurant,Turkish Restaurant,Café,Middle Eastern Restaurant,Lounge,Fast Food Restaurant,American Restaurant,Food Court,Football Stadium
5,Al Barsha First,25.114830,55.191360,1.0,Hotel,Bed & Breakfast,Pakistani Restaurant,Pool,Indian Restaurant,Middle Eastern Restaurant,Restaurant,Grocery Store,Dessert Shop,Turkish Restaurant
6,Al Barsha Second,25.107230,55.204850,1.0,Hotel,Indian Restaurant,Coffee Shop,Ice Cream Shop,Park,Chinese Restaurant,Bakery,Tennis Court,Seafood Restaurant,Gym
7,Al Barsha South First,25.089580,55.234240,2.0,Bakery,Pharmacy,Grocery Store,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop
8,Al Barsha South Second,25.077390,55.242670,4.0,Café,Supermarket,Office,Gym,Yoga Studio,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
9,Al Barsha South Third,25.062290,55.239950,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Changing Datatype of Cluster Label column

In [39]:
dubai_merged['Cluster Labels'] = dubai_merged["Cluster Labels"].astype(int)

In [40]:
dubai_merged

,Community,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,25.283080,55.334350,4,Park,Scenic Lookout,Cafeteria,Bakery,Pizza Place,Hookah Bar,Flower Shop,Fish Market,Flea Market,Food
1,Al Awir First,25.185930,55.541260,4,Café,Yoga Studio,Dance Studio,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
2,Al Awir Second,25.167920,55.543310,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Al Bada,25.218610,55.264060,4,Café,Coffee Shop,Gym / Fitness Center,Restaurant,Beach,Shopping Plaza,Bakery,Boutique,Electronics Store,English Restaurant
4,Al Baraha,25.282800,55.316780,1,Hotel,Indian Restaurant,Turkish Restaurant,Café,Middle Eastern Restaurant,Lounge,Fast Food Restaurant,American Restaurant,Food Court,Football Stadium
5,Al Barsha First,25.114830,55.191360,1,Hotel,Bed & Breakfast,Pakistani Restaurant,Pool,Indian Restaurant,Middle Eastern Restaurant,Restaurant,Grocery Store,Dessert Shop,Turkish Restaurant
6,Al Barsha Second,25.107230,55.204850,1,Hotel,Indian Restaurant,Coffee Shop,Ice Cream Shop,Park,Chinese Restaurant,Bakery,Tennis Court,Seafood Restaurant,Gym
7,Al Barsha South First,25.089580,55.234240,2,Bakery,Pharmacy,Grocery Store,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop
8,Al Barsha South Second,25.077390,55.242670,4,Café,Supermarket,Office,Gym,Yoga Studio,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
9,Al Barsha South Third,25.062290,55.239950,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
dubai_merged.shape

(131, 14)

### Creating map of Dubai community clusters

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dubai_merged['Latitude'], dubai_merged['Longitude'], dubai_merged['Community'], dubai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 0

In [47]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 0, dubai_merged.columns[[0] + list(range(5, dubai_merged.shape[1]))]]

,Community,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Al Nahda Second,Restaurant,Pakistani Restaurant,Market,Yoga Studio,Fast Food Restaurant,French Restaurant,Football Stadium,Food Court,Food & Drink Shop
67,Al Twar Second,Basketball Court,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop
89,Jebel Ali 1,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
90,Jebel Ali 2,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
114,Ras Al Khor Industrial Second,Coffee Shop,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food


#### Community Characterstics:
Most common venues in this cluster are Football Stadiums and small scale food joints such as Fast Food Restaurants, Fried Chicken Joints, Food Courts and Food & Drink Shops.

#### Cluster 1

In [48]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 1, dubai_merged.columns[[0] + list(range(5, dubai_merged.shape[1]))]]

,Community,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Al Baraha,Indian Restaurant,Turkish Restaurant,Café,Middle Eastern Restaurant,Lounge,Fast Food Restaurant,American Restaurant,Food Court,Football Stadium
5,Al Barsha First,Bed & Breakfast,Pakistani Restaurant,Pool,Indian Restaurant,Middle Eastern Restaurant,Restaurant,Grocery Store,Dessert Shop,Turkish Restaurant
6,Al Barsha Second,Indian Restaurant,Coffee Shop,Ice Cream Shop,Park,Chinese Restaurant,Bakery,Tennis Court,Seafood Restaurant,Gym
14,Al Guoz Fourth,Middle Eastern Restaurant,Fast Food Restaurant,Asian Restaurant,Restaurant,Persian Restaurant,Plaza,Chinese Restaurant,Flea Market,Dim Sum Restaurant
20,Al Karama,Restaurant,Pakistani Restaurant,Asian Restaurant,Fast Food Restaurant,Bakery,Chinese Restaurant,Ice Cream Shop,Middle Eastern Restaurant,Ethiopian Restaurant
24,Al Kifaf,Middle Eastern Restaurant,Hotel,Track,Department Store,North Indian Restaurant,Tea Room,Clothing Store,Asian Restaurant,Park
25,Al Mamzar,Restaurant,Farmers Market,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food,Fondue Restaurant
27,Al Mankhool,Park,Department Store,Filipino Restaurant,Chinese Restaurant,Fast Food Restaurant,Fondue Restaurant,Fish Market,Flea Market,Flower Shop
32,Al Muraqqabat,Middle Eastern Restaurant,Chinese Restaurant,Indian Restaurant,Filipino Restaurant,Grocery Store,Asian Restaurant,Café,Hookah Bar,Spa
34,Al Mushrif,Hotel,Fried Chicken Joint,Restaurant,Asian Restaurant,Ice Cream Shop,Middle Eastern Restaurant,Chinese Restaurant,Pakistani Restaurant,Martial Arts Dojo


#### Community Characterstics:
Cluster 1 is very commercial and mostly concetrated towards North Dubai. It mostly consists of a huge varierty of Restaurants, Hotels, Pubs, shops, cafes, restaurants, bars and pubs.

#### Cluster 2

In [49]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 2, dubai_merged.columns[[0] + list(range(5, dubai_merged.shape[1]))]]

,Community,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Al Barsha South First,Pharmacy,Grocery Store,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop
13,Al Garhoud,Park,Grocery Store,Currency Exchange,African Restaurant,Filipino Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
56,Al Rashidiya,Fried Chicken Joint,Burger Joint,Bus Station,Asian Restaurant,Cafeteria,Café,Yoga Studio,Flea Market,French Restaurant
59,Al Safa First,Ice Cream Shop,Tennis Court,Video Store,Italian Restaurant,Grocery Store,Flower Shop,Filipino Restaurant,Fish Market,Flea Market
61,Al Safouh First,Grocery Store,Pharmacy,Playground,Yoga Studio,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant
71,Al Warqa'a Fourth,Grocery Store,Yoga Studio,Filipino Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop
96,Jumeira Third,Café,Ice Cream Shop,Tennis Court,Video Store,Italian Restaurant,Flower Shop,Filipino Restaurant,Fish Market,Flea Market
99,Muhaisanah Fourth,Grocery Store,Gym,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop
102,Muhaisnah First,Hostel,Grocery Store,Yoga Studio,Farmers Market,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
125,Umm Suqeim Third,Fast Food Restaurant,Spa,Breakfast Spot,Diner,Racetrack,Pharmacy,French Restaurant,Sandwich Place,Bakery


#### Community Characterstics:
Cluster 2 is slightly away from the coast and a bit in the interior of Dubai. It mostly consists of small scale businesses such as Grocery Stores, Flea Markets, Fish Markets, Small Restaurants.

#### Cluster 3

In [50]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 3, dubai_merged.columns[[0] + list(range(5, dubai_merged.shape[1]))]]

,Community,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,Dubai Investment Park Second,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food,Fondue Restaurant


#### Community Characterstics:
Cluster 3 is very far from other cluster and lies far inland from a coast or border of Dubai. lt consists of an Investment Park with a few restaurants.

#### Cluster 4

In [51]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 4, dubai_merged.columns[[0] + list(range(5, dubai_merged.shape[1]))]]

,Community,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,Scenic Lookout,Cafeteria,Bakery,Pizza Place,Hookah Bar,Flower Shop,Fish Market,Flea Market,Food
1,Al Awir First,Yoga Studio,Dance Studio,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
3,Al Bada,Coffee Shop,Gym / Fitness Center,Restaurant,Beach,Shopping Plaza,Bakery,Boutique,Electronics Store,English Restaurant
8,Al Barsha South Second,Supermarket,Office,Gym,Yoga Studio,French Restaurant,Football Stadium,Food Court,Food & Drink Shop,Food
10,Al Barsha Third,Art Gallery,Bakery,Convenience Store,Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food
11,Al Buteen,Shopping Mall,Jewelry Store,Art Gallery,Fast Food Restaurant,Pakistani Restaurant,Farmers Market,Flower Shop,Electronics Store,Juice Bar
12,Al Dhagaya,Jewelry Store,Afghan Restaurant,Shopping Mall,Currency Exchange,Electronics Store,Pakistani Restaurant,Clothing Store,Fast Food Restaurant,Flea Market
15,"Al Hamriya, Dubai",Coffee Shop,Café,Indian Restaurant,Asian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Pizza Place,Shopping Mall,Persian Restaurant
16,Al Hamriya Port,Arcade,Cafeteria,Yoga Studio,Filipino Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Court
17,Al Hudaiba,Middle Eastern Restaurant,Café,Cafeteria,Hotel,Convenience Store,Coffee Shop,Chinese Restaurant,Fried Chicken Joint,Bar


#### Community Characterstics:
Cluster 4 is the biggest, very urban and the most commercial area of Dubai with high end Restraunts, Shopping Malls, Yoga Studios, Bars, Cafes.

#### Cluster 5

In [52]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 5, dubai_merged.columns[[0] + list(range(5, dubai_merged.shape[1]))]]

,Community,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Al Awir Second,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Al Barsha South Third,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Al Khwaneej First,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Al Khwaneej Second,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,Al Warqa'a Fifth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,Al Warqa'a Third,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Aleyas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,Jebel Ali Industrial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,Jebel Ali Palm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,Nadd Al Shiba Fourth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Community Characterstics:
Cluster 5 consists of remote areas of Dubai with no venues or attractions and mostly consists of areas in desert.

### Observations

In this project, we went through the process of identifying the business problem, specifying the
data required, extracting and preparing the data, performing machine learning by clustering the
data into 6 clusters based on their similarities, and lastly providing recommendations to the
relevant stakeholders i.e. real estate developers and investors regarding the best locations to open
a new Business Park. To answer the business question that was raised in the introduction section,
the answer proposed by this project is: The communities in Cluster 4 and Cluster1 are the most
preferred locations to open a new Business Park. The findings of this project will help the relevant
stakeholders to capitalize on the opportunities on high potential locations while avoiding the areas
with little commercial value, while making decisions to open a new Business Park.